In [1]:
import os
import tensorflow as tf
import numpy as np
labels_dict = {0:'no_sub', 1:'sub'}
str_to_label = {'sub':1,'no_sub':0}
def create_dataset(folder_path, train_ratio=0.9):
    def process_path(file_path):
        label = get_label(file_path)
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        return img, label

    def get_label(file_path):
        # Extract the label from the file path
        parts = tf.strings.split(file_path, os.path.sep)        
        if parts[-2] == "sub":
            return tf.constant(1, dtype=tf.int32)
        else:
            return tf.constant(0, dtype=tf.int32)

    # Get the list of all image file paths
    sub_dir = os.path.join(folder_path, "sub")
    no_sub_dir = os.path.join(folder_path, "no_sub")

    sub_files = [os.path.join(sub_dir, f) for f in os.listdir(sub_dir) if f.endswith(".jpg")]
    no_sub_files = [os.path.join(no_sub_dir, f) for f in os.listdir(no_sub_dir) if f.endswith(".jpg")]

    all_files = sub_files + no_sub_files
    
    # Shuffle the file paths and split them into training and testing sets
    np.random.shuffle(all_files)
    split_idx = int(len(all_files) * train_ratio)
    train_files = all_files[:split_idx]
    test_files = all_files[split_idx:]

    # Create datasets from the file paths
    train_ds = tf.data.Dataset.from_tensor_slices(train_files)
    test_ds = tf.data.Dataset.from_tensor_slices(test_files)

    # Process the file paths to read and preprocess the images
    train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    return train_ds, test_ds

folder_path = "lenses/lenses/"
train_ds, test_ds = create_dataset(folder_path)

2023-04-01 13:56:59.336180: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 13:56:59.410990: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-01 13:57:00.858443: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-01 13:57:00.884391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_exec

In [2]:
def create_simclr_base_model(input_shape):
    base_model = applications.resnet_v2.ResNet50V2(include_top=False, weights=None, input_shape=input_shape)
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model
def contrastive_loss(z, temperature=0.5):
    batch_size = tf.shape(z)[0]
    z_norm = tf.math.l2_normalize(z, axis=1)
    z_mat = tf.matmul(z_norm, tf.transpose(z_norm))
    positives = tf.linalg.diag_part(z_mat)
    negatives = tf.reshape(z_mat - tf.linalg.diag(tf.ones(batch_size)), [batch_size, -1])
    loss = -tf.math.log(tf.math.exp(positives / temperature) / tf.reduce_sum(tf.math.exp(negatives / temperature), axis=1))
    return tf.reduce_mean(loss)

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, applications


# Data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1)),
])

# Assume train_ds and test_ds are provided as TFDS datasets
# Apply data augmentation to the training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

# Pretrain the model with SimCLR
input_shape = (150, 150, 3)  # Adjust this according to your dataset
batch_size = 16
epochs = 50

simclr_base_model = create_simclr_base_model(input_shape)
simclr_model = models.Sequential([simclr_base_model])
simclr_optimizer = optimizers.Adam(lr=0.001)

@tf.function
def train_step(x):
    with tf.GradientTape() as tape:
        z = simclr_model(x, training=True)
        loss = contrastive_loss(z)
    gradients = tape.gradient(loss, simclr_model.trainable_variables)
    simclr_optimizer.apply_gradients(zip(gradients, simclr_model.trainable_variables))
    return loss

# Self-supervised pre-training with SimCLR
for epoch in range(epochs):
    epoch_loss = 0
    step = 0
    for x_batch, _ in train_ds.batch(batch_size):
        step_loss = train_step(x_batch)
        epoch_loss += step_loss
        step += 1
    epoch_loss /= step
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

2023-04-01 13:54:44.102715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-04-01 13:54:47.150116: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-01 13:54:48.512588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-01 13:54:48.525047: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fe50e35fac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-01 13:54:48.525095: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-04-01 13:54:48.533958: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-01 13:

In [ ]:
def create_classifier(base_model):
    inputs = tf.keras.Input(shape=base_model.output_shape[1:])
    x = layers.Dense(1, activation="sigmoid")(inputs)
    classifier = tf.keras.Model(inputs=inputs, outputs=x)
    return classifier

In [3]:
simclr_base_model.trainable = False

classifier = create_classifier(simclr_base_model)
full_model = models.Sequential([simclr_base_model, classifier])

full_model.compile(optimizer=optimizers.Adam(lr=0.001),
loss="binary_crossentropy",
metrics=["accuracy"])

NameError: name 'simclr_base_model' is not defined